In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
# Initialize models
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import tqdm as notebook_tqdm
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import seaborn as sns
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb

In [2]:
hourly = pd.read_pickle('/Users/sele/Desktop/AUS FALL 24/repositories/Data-Mining-Project/Dataset/hourly.pkl')  # Read from pickle
fixed = pd.read_pickle('/Users/sele/Desktop/AUS FALL 24/repositories/Data-Mining-Project/Dataset/fixed.pkl')

In [3]:
pd.set_option('display.max_columns', 500)
hourly.head()

,Payment_Situation_Payment verified,Hourly_Rate,EX_level_demand,Spent,Connects,Feedback_Num,Applicants_Num,Client_Country_Aland Islands,Client_Country_Albania,Client_Country_Algeria,Client_Country_American Samoa,Client_Country_Andorra,Client_Country_Angola,Client_Country_Anguilla,Client_Country_Argentina,Client_Country_Armenia,Client_Country_Australia,Client_Country_Austria,Client_Country_Azerbaijan,Client_Country_Bahamas,Client_Country_Bahrain,Client_Country_Bangladesh,Client_Country_Barbados,Client_Country_Belgium,Client_Country_Belize,Client_Country_Benin,Client_Country_Bermuda,Client_Country_Bolivia,Client_Country_Bosnia and Herzegovina,Client_Country_Brazil,Client_Country_British Indian Ocean Territory,Client_Country_British Virgin Islands,Client_Country_Brunei Darussalam,Client_Country_Bulgaria,Client_Country_Cambodia,Client_Country_Cameroon,Client_Country_Canada,Client_Country_Cayman Islands,Client_Country_Chile,Client_Country_China,Client_Country_Colombia,Client_Country_Costa Rica,Client_Country_Cote d'Ivoire,Client_Country_Croatia,Client_Country_Curacao,Client_Country_Cyprus,Client_Country_Czech Republic,Client_Country_Denmark,Client_Country_Dominican Republic,Client_Country_Ecuador,Client_Country_Egypt,Client_Country_El Salvador,Client_Country_Estonia,Client_Country_Ethiopia,Client_Country_Finland,Client_Country_France,Client_Country_Gambia,Client_Country_Georgia,Client_Country_Germany,Client_Country_Ghana,Client_Country_Gibraltar,Client_Country_Greece,Client_Country_Greenland,Client_Country_Grenada,Client_Country_Guatemala,Client_Country_Guernsey,Client_Country_Guinea,Client_Country_Haiti,Client_Country_Honduras,Client_Country_Hong Kong,Client_Country_Hungary,Client_Country_Iceland,Client_Country_India,Client_Country_Indonesia,Client_Country_Ireland,Client_Country_Isle of Man,Client_Country_Israel,Client_Country_Italy,Client_Country_Jamaica,Client_Country_Japan,Client_Country_Jersey,Client_Country_Jordan,Client_Country_Kazakhstan,Client_Country_Kenya,Client_Country_Kuwait,Client_Country_Kyrgyzstan,Client_Country_Laos,Client_Country_Latvia,Client_Country_Lebanon,Client_Country_Liechtenstein,Client_Country_Lithuania,Client_Country_Luxembourg,Client_Country_Macao,Client_Country_Macedonia,Client_Country_Malaysia,Client_Country_Maldives,Client_Country_Mali,Client_Country_Malta,Client_Country_Mauritius,Client_Country_Mexico,"Client_Country_Micronesia, Federated States of",Client_Country_Moldova,Client_Country_Monaco,Client_Country_Mongolia,Client_Country_Montenegro,Client_Country_Morocco,Client_Country_Mozambique,Client_Country_Myanmar,Client_Country_Nepal,Client_Country_Netherlands,Client_Country_New Zealand,Client_Country_Nicaragua,Client_Country_Nigeria,Client_Country_Norway,Client_Country_Oman,Client_Country_Pakistan,Client_Country_Palestinian Territories,Client_Country_Panama,Client_Country_Papua New Guinea,Client_Country_Paraguay,Client_Country_Peru,Client_Country_Philippines,Client_Country_Poland,Client_Country_Portugal,Client_Country_Puerto Rico,Client_Country_Qatar,Client_Country_Romania,Client_Country_Russia,Client_Country_Rwanda,Client_Country_Saint Martin (French part),Client_Country_Saint Vincent and the Grenadines,Client_Country_Saudi Arabia,Client_Country_Senegal,Client_Country_Serbia,Client_Country_Seychelles,Client_Country_Singapore,Client_Country_Slovakia,Client_Country_Slovenia,Client_Country_Somalia,Client_Country_South Africa,Client_Country_South Korea,Client_Country_Spain,Client_Country_Sri Lanka,Client_Country_Suriname,Client_Country_Swaziland,Client_Country_Sweden,Client_Country_Switzerland,Client_Country_Taiwan,Client_Country_Tanzania,Client_Country_Thailand,Client_Country_Trinidad and Tobago,Client_Country_Tunisia,Client_Country_Turkey,Client_Country_Turks and Caicos Islands,Client_Country_Uganda,Client_Country_Ukraine,Client_Country_United Arab Emirates,Client_Country_United Kingdom,Client_Country_United States,Client_Country_United States Minor Outlying Islands,Client_Country_United States Virg

In [4]:
hourly.drop(['Feedback_Num'], axis=1, inplace=True)
fixed.drop(['Feedback_Num'], axis=1, inplace=True)

In [10]:
hourly.drop(['Spent'], axis=1, inplace=True)
fixed.drop(['Spent'], axis=1, inplace=True)

In [11]:
from sklearn.preprocessing import MinMaxScaler

# Extract input features
X_embeddings_hourly = np.stack(hourly['row_embedding'].values)  # Convert list of embeddings to a numpy array
# Add numerical features (drop target and embedding columns)
numerical_hourly = hourly.drop(columns=['Hourly_Rate', 'row_embedding'], axis=1)
# Combine embeddings and numerical features
X_hourly = np.hstack((X_embeddings_hourly, numerical_hourly.values))
# Extract the target variable
y_hourly =hourly['Hourly_Rate'].values


# Extract input features
X_embeddings_fixed = np.stack(fixed['row_embedding'].values)  # Convert list of embeddings to a numpy array

# Add numerical features (drop target and embedding columns)
numerical_fixed = fixed.drop(columns=['Job_Cost', 'row_embedding'], axis=1)

# Combine embeddings and numerical features
X_fixed = np.hstack((X_embeddings_fixed, numerical_fixed.values))

# Extract the target variable
y_fixed =fixed['Job_Cost'].values


In [12]:
from sklearn.model_selection import train_test_split

# Split into train and test sets
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_hourly, y_hourly, test_size=0.2, random_state=42)
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_fixed, y_fixed, test_size=0.2, random_state=42)


## XGBOOST

In [13]:
hourly_model = XGBRegressor()
fixed_model = XGBRegressor()

hourly_model.fit(X_train_h, y_train_h)
fixed_model.fit(X_train_f, y_train_f)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
# Predictions
hourly_preds = hourly_model.predict(X_test_h)
fixed_preds = fixed_model.predict(X_test_f)

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

def evaluate(y_test, predictions, normalization='range'):
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    medae = median_absolute_error(y_test, predictions)  # Calculate MedAE
    r2 = r2_score(y_test, predictions)
    rmse = np.sqrt(mse)  # Calculate RMSE
    
    if normalization == 'range':
        # NRMSE using range
        nrmse = rmse / (y_test.max() - y_test.min())
    elif normalization == 'mean':
        # NRMSE using mean
        nrmse = rmse / y_test.mean()
    else:
        raise ValueError("Invalid normalization method. Use 'range' or 'mean'.")
    
    return mse, mae, medae, r2, rmse, nrmse

# Evaluate hourly model
hourly_metrics = evaluate(y_test_h, hourly_preds, normalization='range')
print(f"Hourly Model - MSE: {hourly_metrics[0]}, MAE: {hourly_metrics[1]}, MedAE: {hourly_metrics[2]}, R2: {hourly_metrics[3]}, RMSE: {hourly_metrics[4]}, NRMSE: {hourly_metrics[5]}")

# Evaluate fixed model
fixed_metrics = evaluate(y_test_f, fixed_preds, normalization='range')
print(f"Fixed Model - MSE: {fixed_metrics[0]}, MAE: {fixed_metrics[1]}, MedAE: {fixed_metrics[2]}, R2: {fixed_metrics[3]}, RMSE: {fixed_metrics[4]}, NRMSE: {fixed_metrics[5]}")

Hourly Model - MSE: 324.79503705563263, MAE: 13.410458964373875, MedAE: 10.424956321716309, R2: 0.10557554795531376, RMSE: 18.02207083150082, NRMSE: 0.15403479343163093
Fixed Model - MSE: 2560980.0194008374, MAE: 706.6999161471017, MedAE: 244.92300415039062, R2: 0.15977470905902202, RMSE: 1600.306226758128, NRMSE: 0.10672265600254272


## CatBoost

In [17]:
# Define CatBoost Regressor
model_hourly = CatBoostRegressor(
    iterations=500,  # Number of boosting iterations
    learning_rate=0.1,  # Learning rate
    depth=8,  # Depth of the trees
    loss_function='RMSE',  # Loss function
    verbose=100  # Print training progress every 100 iterations
)

# Train the model for hourly rate prediction
model_hourly.fit(
    X_train_h, y_train_h,
    eval_set=(X_test_h, y_test_h),
    plot=True,  # Plot loss during training
    early_stopping_rounds=50  # Stop training if validation loss does not improve for 50 rounds
)

# Predict on test set
y_pred_hourly = model_hourly.predict(X_test_h)

# Evaluation metrics
mse_hourly = mean_squared_error(y_test_h, y_pred_hourly)
rmse_hourly = np.sqrt(mse_hourly)
mae_hourly = mean_absolute_error(y_test_h, y_pred_hourly)
r2_hourly = r2_score(y_test_h, y_pred_hourly)

# Print evaluation results
print(f"Hourly Rate Prediction:")
print(f"MSE: {mse_hourly:.4f}")
print(f"RMSE: {rmse_hourly:.4f}")
print(f"MAE: {mae_hourly:.4f}")
print(f"R^2 Score: {r2_hourly:.4f}")

# Repeat for fixed price prediction
model_fixed = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=8,
    loss_function='RMSE',
    verbose=100
)

model_fixed.fit(
    X_train_f, y_train_f,
    eval_set=(X_test_f, y_test_f),
    plot=True,
    early_stopping_rounds=50
)

y_pred_fixed = model_fixed.predict(X_test_f)

mse_fixed = mean_squared_error(y_test_f, y_pred_fixed)
rmse_fixed = np.sqrt(mse_fixed)
mae_fixed = mean_absolute_error(y_test_f, y_pred_fixed)
r2_fixed = r2_score(y_test_f, y_pred_fixed)

print(f"\nFixed Price Prediction:")
print(f"MSE: {mse_fixed:.4f}")
print(f"RMSE: {rmse_fixed:.4f}")
print(f"MAE: {mae_fixed:.4f}")
print(f"R^2 Score: {r2_fixed:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 19.2088748	test: 18.8729214	best: 18.8729214 (0)	total: 254ms	remaining: 2m 6s
100:	learn: 17.5719917	test: 17.7709873	best: 17.7694956 (96)	total: 7.05s	remaining: 27.9s
200:	learn: 17.1540035	test: 17.7139800	best: 17.7138208 (187)	total: 13.2s	remaining: 19.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 17.70969503
bestIteration = 207

Shrink model to first 208 iterations.
Hourly Rate Prediction:
MSE: 313.6333
RMSE: 17.7097
MAE: 13.2035
R^2 Score: 0.1363


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1675.8272781	test: 1712.3261776	best: 1712.3261776 (0)	total: 70.1ms	remaining: 35s
100:	learn: 1432.3999482	test: 1550.4173049	best: 1549.3779053 (84)	total: 3.97s	remaining: 15.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 1549.377905
bestIteration = 84

Shrink model to first 85 iterations.

Fixed Price Prediction:
MSE: 2400571.8939
RMSE: 1549.3779
MAE: 681.4784
R^2 Score: 0.2124


## LightGBM

In [ ]:
train_data_hourly = lgb.Dataset(X_train_h, label=y_train_h)
test_data_hourly = lgb.Dataset(X_test_h, label=y_test_h, reference=train_data_hourly)

train_data_fixed = lgb.Dataset(X_train_f, label=y_train_f)
test_data_fixed = lgb.Dataset(X_test_f, label=y_test_f, reference=train_data_fixed)

# Set LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',  # Gradient Boosted Decision Trees
    'learning_rate': 0.1,
    'num_leaves': 31,  # Maximum number of leaves in one tree
    'max_depth': -1,   # No limit on tree depth
    'verbose': -1
}

# Train LightGBM model for hourly rate prediction
print("Training LightGBM for Hourly Rate Prediction...")
model_hourly = lgb.train(
    params,
    train_data_hourly,
    valid_sets=[train_data_hourly, test_data_hourly],
    num_boost_round=500  # Stop training if validation score doesn't improve for 50 rounds
    
)

# Predict on test set
y_pred_hourly = model_hourly.predict(X_test_h)

# Evaluate performance
mse_hourly = mean_squared_error(y_test_h, y_pred_hourly)
rmse_hourly = np.sqrt(mse_hourly)
mae_hourly = mean_absolute_error(y_test_h, y_pred_hourly)
r2_hourly = r2_score(y_test_h, y_pred_hourly)

print(f"\nHourly Rate Prediction:")
print(f"MSE: {mse_hourly:.4f}")
print(f"RMSE: {rmse_hourly:.4f}")
print(f"MAE: {mae_hourly:.4f}")
print(f"R^2 Score: {r2_hourly:.4f}")

# Train LightGBM model for fixed price prediction
print("\nTraining LightGBM for Fixed Price Prediction...")
model_fixed = lgb.train(
    params,
    train_data_fixed,
    valid_sets=[train_data_fixed, test_data_fixed],
    num_boost_round=500
    
)

# Predict on test set
y_pred_fixed = model_fixed.predict(X_test_f)

# Evaluate performance
mse_fixed = mean_squared_error(y_test_f, y_pred_fixed)
rmse_fixed = np.sqrt(mse_fixed)
mae_fixed = mean_absolute_error(y_test_f, y_pred_fixed)
r2_fixed = r2_score(y_test_f, y_pred_fixed)

print(f"\nFixed Price Prediction:")
print(f"MSE: {mse_fixed:.4f}")
print(f"RMSE: {rmse_fixed:.4f}")
print(f"MAE: {mae_fixed:.4f}")
print(f"R^2 Score: {r2_fixed:.4f}")



In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error


# Custom scorer for RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Define LightGBM model
lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', metric='rmse')

# Define parameter grid for randomized search
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],  # Learning rates to try
    'num_leaves': [10, 20, 31, 50, 100],          # Number of leaves
    'max_depth': [-1, 5, 10, 15, 20],             # Max depth of trees
    'n_estimators': [100, 200, 500]               # Number of boosting rounds
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring=rmse_scorer,  # Use RMSE as the evaluation metric
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available CPU cores
)

# Fit RandomizedSearchCV to the data (example with hourly rate prediction)
random_search.fit(X_train_h, y_train_h)

# Print the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("Best RMSE:", -random_search.best_score_)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018946 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27124
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.103739 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.108108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28572
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.104928 seconds.
You can set `force_col_wise=true` to remove the overhead.[LightGBM] [Info] Total Bins 30393
[LightGBM] [Info] Number of data points in the train set: 14133, number of used features: 361

[LightGBM] [Info] Total Bin